In [ ]:
#!export CUDA_VISIBLE_DEVICES=1

In [1]:
import numpy as np

import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

#import tensorflow_addons as tfa

print(tf.__version__)


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#%%

folder = os.path.join('..','..','short_data_4_model','20')


data_train = np.load(os.path.join(folder, 'train_data.npz'),'r')
data_train = data_train['arr_0']

data_valid = np.load(os.path.join(folder, 'valid_data.npz'),'r')
data_valid = data_valid['arr_0']
'''
data_test = np.load(os.path.join(folder, 'test_data.npz'),'r')
data_test = data_test['arr_0']
'''
print('data loaded successfully')

2.0.0
Num GPUs Available:  0
data loaded successfully


In [2]:
EPOCHS = 15
BATCH_SIZE =60
N_model = 5
units = 100
# no use
embedding_dim = 256
seq_len =50
num_examples = len(data_train)
steps_per_epoch = num_examples//BATCH_SIZE


dataset = tf.data.Dataset.from_tensor_slices(data_train).shuffle(num_examples)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
drop_out =0.2

In [3]:
class Encoder(tf.keras.Model):
    def __init__(self, encoding_units, batch_size, dropout):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoding_units = encoding_units
        self.dropout = dropout
        self.gru = keras.layers.GRU(self.encoding_units,
                                     return_sequences=True,
                                     return_state=True,
                                     dropout=self.dropout, 
                                     recurrent_dropout=0.0,
                                     recurrent_initializer='glorot_uniform')

    def call(self, x,h):

        output, h = self.gru(x, initial_state = h)
        return output, h
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size,self.encoding_units)),tf.zeros((self.batch_size,self.encoding_units))

    
class Encoder_wrap(tf.keras.Model):
    def __init__(self, encoding_units, batch_size, dropout):
        super(Encoder_wrap, self).__init__()
        self.batch_size = batch_size
        self.encoding_units = encoding_units
        self.dropout = dropout
        self.en_1 = Encoder(self.encoding_units, self.batch_size, self.dropout)
        self.en_2 = Encoder(self.encoding_units, self.batch_size, self.dropout)
        self.en_3 = Encoder(self.encoding_units, self.batch_size, self.dropout)
        self.en_4 = Encoder(self.encoding_units, self.batch_size, self.dropout)
        self.en_5 = Encoder(self.encoding_units, self.batch_size, self.dropout)

        self.do_h = keras.layers.Dropout(self.dropout)

        self.fc_h = keras.layers.Dense(self.encoding_units)
    
    def call(self, x):
        m,h = self.en_1.initialize_hidden_state()
        _, h_1 = self.en_1(x,h)
        _, h_2 = self.en_1(x,h)
        _, h_3 = self.en_1(x,h)
        _, h_4 = self.en_1(x,h)
        _, h_5 = self.en_1(x,h)

        concatted_h = tf.keras.layers.Concatenate()([h_1, h_2, h_3, h_4, h_5])
        
        op_h = self.do_h(concatted_h)
        op_h = self.fc_h(op_h)

        
        return op_h
        
        
example_input_batch = np.zeros((BATCH_SIZE,seq_len,3))
# sample input

encoder = Encoder_wrap(units, BATCH_SIZE,drop_out)

h = encoder(example_input_batch)

print('Encoder Hidden state shape: (batch size, units) {}'.format(h.shape))





If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Encoder Hidden state shape: (batch size, units) (60, 100)


In [4]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, decoding_units, batch_size, dropout):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.decoding_units = decoding_units
        self.dropout = dropout
        #self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.decoding_units,
                                      return_sequences=True,
                                      return_state=True,
                                      dropout=self.dropout, 
                                      recurrent_dropout=0.0,
                                      recurrent_initializer='glorot_uniform')
        self.fc = keras.layers.Dense(vocab_size)
        self.do_m = keras.layers.Dropout(self.dropout)
        self.do_h = keras.layers.Dropout(self.dropout)
        
    def call(self, x, h):
        h= self.do_h(h)



        rnn, h_o = self.gru(x,initial_state = h)

        op = self.fc(rnn)

        return op, h_o


class Decoder_wrap(tf.keras.Model):
    def __init__(self, vocab_size, decoding_units, batch_size, dropout):
        super(Decoder_wrap, self).__init__()
        self.batch_size = batch_size
        self.decoding_units = decoding_units
        self.dropout = dropout
        self.vocab_size = vocab_size
        self.de_0 = Decoder(self.vocab_size,self.decoding_units, self.batch_size, self.dropout )
        self.de_1 = Decoder(self.vocab_size,self.decoding_units, self.batch_size, self.dropout )
        self.de_2 = Decoder(self.vocab_size,self.decoding_units, self.batch_size, self.dropout )
        self.de_3 = Decoder(self.vocab_size,self.decoding_units, self.batch_size, self.dropout )
        self.de_4 = Decoder(self.vocab_size,self.decoding_units, self.batch_size, self.dropout )
    
    def call(self, x, h):
        op_0, h_0 = self.de_1(x[0],  h[0])
        op_1, h_1 = self.de_1(x[1],  h[1])
        op_2, h_2 = self.de_1(x[2],  h[2])
        op_3, h_3 = self.de_1(x[3],  h[3])
        op_4, h_4 = self.de_1(x[4],  h[4])
        
        op =tf.keras.layers.Concatenate(axis=0)([tf.expand_dims(op_0,0), tf.expand_dims(op_1,0), 
                                           tf.expand_dims(op_2,0), tf.expand_dims(op_3,0),
                                           tf.expand_dims(op_4,0)])
        
        op_h = tf.keras.layers.Concatenate(axis=0)([tf.expand_dims(h_0,0), tf.expand_dims(h_1,0),
                                              tf.expand_dims(h_2,0), tf.expand_dims(h_3,0),
                                              tf.expand_dims(h_4,0)])
        return op, op_h
        
        
                        
vocab_size=3
decoder = Decoder_wrap(vocab_size,units, BATCH_SIZE, drop_out)
m = tf.random.uniform((5,BATCH_SIZE,units))
sample_decoder_output,  de_h = decoder(tf.random.uniform((5, BATCH_SIZE, 1,3)),m)
print ('Decoder output shape: (5,batch_size,1, vocab size) {}'.format(sample_decoder_output.shape))


Decoder output shape: (5,batch_size,1, vocab size) (5, 60, 1, 3)


In [5]:
optimizer = keras.optimizers.Adam()
loss_object = tf.keras.losses.MeanSquaredError( reduction=tf.keras.losses.Reduction.NONE)

@tf.function
def loss_function(real, pred):
    
    #mask = tf.math.logical_not(tf.math.equal(real, [0,0,0]))
    loss_ = loss_object(real, pred)

    #mask = tf.cast(mask, dtype=loss_.dtype)
    #loss_ *= mask

    return tf.reduce_mean(loss_)

checkpoint_dir = str(seq_len)+'full_seq2seq-sf-GRU-simple_'+str(units)+'_'+str(BATCH_SIZE)
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)



In [7]:
def expand_n_concat(ip,n):
    ans = tf.expand_dims(ip,0)
    for i in range(n-1):
        tmp = tf.expand_dims(ip,0)
        ans = tf.keras.layers.Concatenate(axis=0,dtype=tf.float32)([ans,tmp])
    return ans

In [8]:
def evalute_batch(inputs):
    result = np.zeros((np.shape(inputs)))
    inp = tf.convert_to_tensor(inputs)

    inference_batch_size = inp.shape[0]
    h_en= encoder(inp)
    de_in = tf.expand_dims(inp[:,0,:], 1)
    de_in = expand_n_concat(de_in, N_model)

    h_en = expand_n_concat(h_en, N_model)
    for t in range(inp.shape[1]):
        predictions, h_en = decoder(de_in, h_en)
        tmp = np.mean(predictions,axis=0)
        tmp = tf.squeeze(tmp)
        result[:,t,:] = tmp
        de_in = predictions
    return result

optimizer = keras.optimizers.Adam()
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [9]:
def evalute_subj(inputs):
    result = np.zeros((np.shape(inputs)))
    bz = BATCH_SIZE
    num = np.shape(inputs)[0] // BATCH_SIZE
    for i in range(num):
        in_tmp = inputs[i*bz:(i+1)*bz]
        res_tmp = evalute_batch(in_tmp)
        result[i*bz:(i+1)*bz] = res_tmp
        
    if np.shape(inputs)[0] % BATCH_SIZE:
        rem = np.shape(inputs)[0] % BATCH_SIZE
        in_tmp = np.zeros((np.shape(res_tmp)))
        in_tmp[0:rem] = inputs[num*bz:]
        res_tmp = evalute_batch(in_tmp)
        result[num*bz:] = res_tmp[0:rem]
    return result

In [10]:
from sklearn.metrics import mean_squared_error
def cal_mse(data_raw,data_pred):
    mse = np.zeros(np.shape(data_raw)[0])
    for i in range(np.shape(data_raw)[0]):
        result = mean_squared_error(data_raw[i], data_pred[i])
        mse[i] = result
    return mse  

In [17]:
from plyfile import PlyData


from sklearn.metrics import mean_squared_error
from dipy.tracking.streamline import Streamlines
from dipy.tracking.streamlinespeed import length
                                          

class PlyStruct:
    """Class that process poly data"""

    def __init__(self):
        self.ply_data = None
        self.idx = None
        self.properties = None
        self.stream_line = None

    def load_poly_data(self, subpath: str, num_prop: int = None):
        """
        Load ply file
        :param str subpath: path of ply file
        :param int num_prop: optional parmeter to only take the first num properties (e.g. only x,y,z coordinates)
        """
        ply = PlyData.read(subpath)
        property_list = [ply['vertices'].data[name] for name in ply['vertices'].data.dtype.names]
        self.ply_data = np.array(property_list).T
        if num_prop:
            self.ply_data = self.ply_data[:, :num_prop]
        self.idx = ply['fiber'].data['endindex']
        self.properties = ply['vertices'].data.dtype.names
        if num_prop:
            self.properties = self.properties[:num_prop]
   

    def gen_stream_line(self):
        self.stream_line = []
        self.stream_line.append(self.ply_data[0:self.idx[0],:])
        for i in range(len(self.idx)-1):
            self.stream_line.append(self.ply_data[self.idx[i]:self.idx[i+1],:])


def zero_remove(darray):
    
    for i in range(np.shape(darray)[0]-1,-1,-1):
        if not (np.around(darray[i], decimals=0) == 0 ).all():
            return darray[0:i+1] 
            break


def get_length(stream):
    # to get the length of streams in this subject
    len_table = []
    for i in range(len(stream)):
        len_table.append(np.shape(stream[i])[0])
    return len_table

def cal_mse(data_raw,data_pred):
    mse = np.zeros(np.shape(data_raw)[0])
    for i in range(np.shape(data_raw)[0]):
        result = mean_squared_error(data_raw[i], data_pred[i])
        mse[i] = result
    return mse  


def ply2np(name):
    # convert a ply format to the matrix we are using
    # input : '128127_ex_cc-body_shore.ply', name of a oly file
    # optput: a matrix (#_of_fibers, #_of_vertex, 3 )
    temo = PlyStruct()
    temo.load_poly_data(os.path.join('..','data',name),num_prop=3)
    #data = temo.ply_data
  
    temo.gen_stream_line()
    stream = temo.stream_line


    stream = np.array(stream)
    len_table = get_length(stream)
    subject = np.zeros((len(stream),np.max(len_table),3))
    for i in range(len(stream)):
        length = len_table[i]
        subject[i,0:length,:] = stream[i]
        
    return subject

from sklearn.metrics import mean_squared_error
def cal_mse(data_raw,data_pred):
    mse = np.zeros(np.shape(data_raw)[0])
    for i in range(np.shape(data_raw)[0]):
        result = mean_squared_error(data_raw[i], data_pred[i])
        mse[i] = result
    return mse  

In [42]:
# A way to calculate the validate error
'''
for i in range(10):
    f = open(os.path.join(checkpoint_dir,'checkpoint'), 'w')
    f.write('model_checkpoint_path: "ckpt-'+str(i+1)+'"\n')
    f.write('all_model_checkpoint_paths: "ckpt-'+str(i+1)+'"\n')
    f.close()
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    
    result = evalute_subj(data_valid)
    mse_idv =loss_function(data_valid, result)
    print(str(i+1)+': '+str(mse_idv.numpy()))
'''

1: 13.80529398659342
2: 10.966595235578795
3: 10.344658584767998
4: 11.02308766422285
5: 8.813155492361354
6: 9.53661526227147
7: 14.774001076786943
8: 11.111364702306044
9: 10.661900548514183
10: 8.142202071792822


# type in the parameters
name : the file name\
len_thre: length control value, length below it will be considered abnomal fiber\
thre_con: control value of threshold percentile


In [20]:
name = '156031_ex_cc-body_shore.ply'
len_thre = 40
thre_con = 85

In [22]:
bundle = ply2np(name)

from dipy.tracking.streamlinespeed import (compress_streamlines, length,
                                           set_number_of_points)
streamlines_evl = Streamlines()
for j in range(np.shape(bundle)[0]):
    tmp = bundle[j]
    tmp = zero_remove(tmp)
    streamlines_evl.append(tmp)


subsamp_sls = set_number_of_points(streamlines_evl, seq_len)

data_test= np.array(subsamp_sls)
result = evalute_subj(data_test)

mse_idv = cal_mse(data_test, result)


In [24]:
thre = np.percentile(mse_idv,thre_con)
pred_1 = 1*(mse_idv<thre)


bundle_str = Streamlines()

for i in range(np.shape(bundle)[0]):
    tmp = bundle[i]
    tmp = zero_remove(tmp)
    #tmp = tmp[~np.all(tmp == 0, axis=-1)]
    #tmp = np.around(tmp, decimals=0)
    bundle_str.append(tmp)
    
lengths = length(bundle_str)

pred_2 = 1*(lengths > len_thre)

pred = 1*((pred_1 + pred_2)>1)

op_path = 'result'
if not os.path.exists(op_path):
    os.mkdir(op_path)

save_path = os.path.join(op_path,'Detection'+name.split('_')[0]+'_ens_seq')
np.save(save_path,pred)